In [1]:
#encoding=utf-8
import torch
import torchvision as ptv
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [2]:
train_set = ptv.datasets.MNIST("./mnist/train",train=True,transform=ptv.transforms.ToTensor(),download=False)
test_set = ptv.datasets.MNIST("./mnist/test",train=False,transform=ptv.transforms.ToTensor(),download=False)
train_dataset = torch.utils.data.DataLoader(train_set,batch_size=100)
test_dataset = torch.utils.data.DataLoader(test_set,batch_size=100)

In [3]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet,self).__init__()
        
        self.cnn = nn.Conv2d(in_channels=1,
                            out_channels=16,
                            kernel_size=(3,3),
                            stride=1,
                            padding=1)   #(kernerl_size-1)/2,输出:16,28,28
        self.max_pool = nn.MaxPool2d(kernel_size=(2,2))   # 输出:16,14,14
        #
        self.cnn2 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                     out_channels=32,
                     kernel_size=5,
                     stride=1,
                     padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2) #32,7,7
        )#32,
        
        self.hid = nn.Linear(32*7*7,32)
        
        self.out = nn.Linear(32,10)
    def forward(self,inputs):
        batch_size = inputs.size(0)
        cnn_out = F.relu(self.cnn(inputs))
        pool_out = self.max_pool(cnn_out)
        cnn2_out = self.cnn2(pool_out)
        pool_out = cnn2_out.view(batch_size,-1)
        h_o = F.relu(self.hid(pool_out))
        
        out = self.out(h_o)
        return F.softmax(out)
        
        

In [4]:
model = CNNNet()
print(model)

CNNNet (
  (cnn): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (cnn2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (hid): Linear (1568 -> 32)
  (out): Linear (32 -> 10)
)


In [5]:
optimizer = optim.Adam(model.parameters()，0.003)
criterion = nn.CrossEntropyLoss()

In [7]:
from torch.autograd import Variable
for epoch in range(10):

    for i,(b_x,b_y) in enumerate(train_dataset):
        b_x = Variable(b_x)
        b_y = Variable(b_y)
#         print(b_x.size())
        predict = model(b_x)
        loss = criterion(predict,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss: {:.4f}' 
                   .format(epoch+1, 10, i+1, loss.data[0]))
        

Epoch [1/10], Step [100], Loss: 1.4620
Epoch [1/10], Step [200], Loss: 1.4615
Epoch [1/10], Step [300], Loss: 1.4661
Epoch [1/10], Step [400], Loss: 1.4694
Epoch [1/10], Step [500], Loss: 1.4795
Epoch [1/10], Step [600], Loss: 1.4712
Epoch [2/10], Step [100], Loss: 1.4612
Epoch [2/10], Step [200], Loss: 1.4639
Epoch [2/10], Step [300], Loss: 1.4755
Epoch [2/10], Step [400], Loss: 1.4645
Epoch [2/10], Step [500], Loss: 1.4787
Epoch [2/10], Step [600], Loss: 1.4712
Epoch [3/10], Step [100], Loss: 1.4630
Epoch [3/10], Step [200], Loss: 1.4618
Epoch [3/10], Step [300], Loss: 1.4613
Epoch [3/10], Step [400], Loss: 1.4612
Epoch [3/10], Step [500], Loss: 1.4746
Epoch [3/10], Step [600], Loss: 1.4712
Epoch [4/10], Step [100], Loss: 1.4655
Epoch [4/10], Step [200], Loss: 1.4613
Epoch [4/10], Step [300], Loss: 1.4686
Epoch [4/10], Step [400], Loss: 1.4641
Epoch [4/10], Step [500], Loss: 1.4784
Epoch [4/10], Step [600], Loss: 1.4712
Epoch [5/10], Step [100], Loss: 1.4612
Epoch [5/10], Step [200],

In [12]:
model.eval()
total=0
correct = 0
for i,(b_x,b_y) in enumerate(test_dataset):
    b_x = Variable(b_x)
    outputs = model(b_x)
    _,predict = torch.max(outputs.data,1)
    total += b_y.size(0)
    correct += (predict==b_y).sum()
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98 %
